In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hutchinson-Gilford_Progeria_Syndrome"
cohort = "GSE84351"

# Input paths
in_trait_dir = "../../input/GEO/Hutchinson-Gilford_Progeria_Syndrome"
in_cohort_dir = "../../input/GEO/Hutchinson-Gilford_Progeria_Syndrome/GSE84351"

# Output paths
out_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/GSE84351.csv"
out_gene_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/gene_data/GSE84351.csv"
out_clinical_data_file = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/clinical_data/GSE84351.csv"
json_path = "../../output/preprocess/Hutchinson-Gilford_Progeria_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from typing import Optional, Callable, Dict, Any, List

# Check for gene expression availability
# This is gene expression data from Affymetrix platform, so it likely contains gene expression data
is_gene_available = True

# Define rows for trait, age, and gender
trait_row = 2  # 'condition: HGPS' or 'condition: Normal'
age_row = None  # Age data not available
gender_row = 0  # 'Sex: Male', 'Sex: Female', 'Sex: ?'

# Define conversion functions for each variable
def convert_trait(value):
    """Convert HGPS trait status to binary format."""
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return None
    
    # Convert to string to ensure we can work with it
    value = str(value)
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'hgps':
        return 1  # HGPS positive
    elif value.lower() == 'normal':
        return 0  # HGPS negative
    else:
        return None  # Unknown value

def convert_gender(value):
    """Convert gender to binary format: female=0, male=1."""
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return None
    
    # Convert to string to ensure we can work with it
    value = str(value)
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'male':
        return 1
    elif value.lower() == 'female':
        return 0
    else:
        return None  # Unknown or missing value

# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If trait data is available, extract clinical features
if is_trait_available:
    # Create a DataFrame from the sample characteristics dictionary
    sample_characteristics = {
        0: ['Sex: Male', 'Sex: Female', 'Sex: ?'], 
        1: ['cell line: HGADFN003', 'cell line: HGMDFN090', 'cell line: HGADFN167', 'cell line: HGFDFN168', 'cell line: AG01972', 'cell line: BJ1', 'cell line: H9'], 
        2: ['condition: HGPS', 'condition: Normal'], 
        3: ['cell type: iPSC', 'cell type: Vascular Smooth Muscle', 'cell type: Fibroblast', 'cell type: Embryonic Stem Cell']
    }
    
    # Determine the number of samples (use max length of values in any row)
    max_samples = max(len(values) for values in sample_characteristics.values())
    
    # Create a properly formatted clinical DataFrame 
    # The format expected by geo_select_clinical_features is:
    # - Rows represent features (like trait, gender)
    # - Columns represent samples
    clinical_data = pd.DataFrame(index=range(len(sample_characteristics)), 
                                columns=[f'Sample_{i+1}' for i in range(max_samples)])
    
    # Fill in the DataFrame with the available values
    for row_idx, values in sample_characteristics.items():
        for col_idx, value in enumerate(values):
            if col_idx < max_samples:
                clinical_data.iloc[row_idx, col_idx] = value
            
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save the extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 4: Gene Identifier Review

In [ ]:
# From the gene identifiers in the previous step, these appear to be numeric IDs (like 16650001, 16650003, etc.)
# rather than standard human gene symbols. Human gene symbols would typically be alphabetic characters 
# (like BRCA1, TP53, LMNA, etc.).
# 
# These numeric IDs likely refer to probe or feature IDs from a microarray platform and would need
# to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Observe the gene identifiers and decide which columns to use for mapping
print("Analyzing gene identifiers and annotation data...")

# From the previous steps we can see:
# - Gene expression data uses numeric IDs (like 16650001) in the 'ID' column
# - Gene annotation has the same 'ID' column
# - 'GB_ACC' contains gene accession numbers that we need to map to gene symbols

# Check if the SPOT_ID column contains genomic coordinates we can use
print("\nAnalyzing SPOT_ID format for genomic locations:")
spot_id_samples = gene_annotation['SPOT_ID'].dropna().unique()[:5]
print(f"Sample SPOT_ID values: {spot_id_samples}")

# GenBank accessions might be useful even if they're non-coding RNAs
# Let's examine the accessions more closely
print("\nAnalyzing GenBank accessions:")
gb_acc_samples = gene_annotation['GB_ACC'].dropna().sample(min(10, gene_annotation['GB_ACC'].count())).tolist()
print(f"Sample GenBank accessions: {gb_acc_samples}")

# 2. Create a custom mapping function for GenBank accessions
# Since our extract_human_gene_symbols function filters out NR_/XR_ accessions,
# we'll create a modified approach that keeps these identifiers
def extract_gene_identifiers(text):
    """Extract gene identifiers from GenBank accessions including non-coding RNAs."""
    if not isinstance(text, str):
        return []
    # Keep the accession ID as is since we don't have proper gene symbols
    return [text]

# Create gene mapping dataframe using ID and GB_ACC columns
prob_col = 'ID'
gene_col = 'GB_ACC'

print(f"\nCreating gene mapping with {prob_col} and {gene_col}...")
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Generated mapping for {len(gene_mapping)} entries")

# Modify the mapping to use our custom extraction function
gene_mapping['Gene'] = gene_mapping['Gene'].apply(extract_gene_identifiers)

# Preview the mapping
print("\nGene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe measurements to gene expression
print("\nConverting probe measurements to gene expression data...")
try:
    # Create a subclass of the mapping function that uses our custom extraction
    from tools.preprocess import apply_gene_mapping as original_apply_gene_mapping
    
    def custom_apply_gene_mapping(expression_df, mapping_df):
        """Modified version that preserves GenBank accessions"""
        mapping_df = mapping_df[mapping_df['ID'].isin(expression_df.index)].copy()
        # Use the already-processed Gene column (from extract_gene_identifiers)
        
        # Count genes per probe and expand to one gene per row
        mapping_df['num_genes'] = mapping_df['Gene'].apply(len)
        mapping_df = mapping_df.explode('Gene')
        # Empty list becomes NaN after explode, which should be dropped
        mapping_df = mapping_df.dropna(subset=['Gene'])
        mapping_df.set_index('ID', inplace=True)
        
        # Merge and distribute expression values
        merged_df = mapping_df.join(expression_df)
        expr_cols = [col for col in merged_df.columns if col not in ['Gene', 'num_genes']]
        merged_df[expr_cols] = merged_df[expr_cols].div(merged_df['num_genes'].replace(0, 1), axis=0)
        
        # Sum expression values for each gene
        gene_expression_df = merged_df.groupby('Gene')[expr_cols].sum()
        
        return gene_expression_df
    
    # Apply our custom mapping function
    gene_data_mapped = custom_apply_gene_mapping(gene_data, gene_mapping)
    
    # Check if we got any results
    if len(gene_data_mapped) > 0:
        gene_data = gene_data_mapped
        print(f"Successfully mapped to {len(gene_data)} gene identifiers")
    else:
        print("Mapping produced no results. Using original gene data with numeric IDs.")
        # Keep the original gene_data 
except Exception as e:
    print(f"Error in gene mapping: {e}")
    print("Using original gene data with numeric IDs.")

# Preview the gene expression data
print("\nGene expression data preview:")
print(f"Gene data shape: {gene_data.shape}")
if len(gene_data) > 0:
    print(preview_df(gene_data))

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")